# Assignment 5
The script should not give an error. Any mistake will be consider as 0.


In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [2]:
#!pip install selenium
#!pip install webdriver-manager

In [3]:
# this library is to manipulate browser
from selenium import webdriver

# it allows you to work with differen versions of drivers
# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time 
from selenium.webdriver.common.by import By

In [4]:
pwd

'C:\\Users\\Alvaro\\Documents\\GitHub\\Diplomado_PUCP\\Lecture_7\\Assignment_5'

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

service = Service(executable_path="../chromedriver-win64/chromedriver.exe")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
url = 'https://infogob.jne.gob.pe/Eleccion'
driver.get(url)

In [18]:
import pandas as pd # for table import

In [25]:
proceso = driver.find_element( By.XPATH  , '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
proceso.click()

presidencial = driver.find_element( By.XPATH  , '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
presidencial.click()

eleccion = driver.find_element( By.XPATH , '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
eleccion.click()

periodo_26 = driver.find_element( By.XPATH , '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[26]')
periodo_26.click()

buscar = driver.find_element( By.XPATH , '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
buscar.click()

resultados = driver.find_element( By.XPATH , '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
resultados.click()

table_path = driver.find_element( By.XPATH , ' ') # HTML DE TABLA ACA
table_html = table_path.get_attribute( 'innerHTML' )


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[2]/table/tbody/tr[1]/td[1]"}
  (Session info: chrome=120.0.6099.217); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF793802142+3514994]
	(No symbol) [0x00007FF793420CE2]
	(No symbol) [0x00007FF7932C76AA]
	(No symbol) [0x00007FF793311860]
	(No symbol) [0x00007FF79331197C]
	(No symbol) [0x00007FF793354EE7]
	(No symbol) [0x00007FF79333602F]
	(No symbol) [0x00007FF7933528F6]
	(No symbol) [0x00007FF793335D93]
	(No symbol) [0x00007FF793304BDC]
	(No symbol) [0x00007FF793305C64]
	GetHandleVerifier [0x00007FF79382E16B+3695259]
	GetHandleVerifier [0x00007FF793886737+4057191]
	GetHandleVerifier [0x00007FF79387E4E3+4023827]
	GetHandleVerifier [0x00007FF7935504F9+689705]
	(No

In [ ]:
table = pd.read_html( table_html )
table[0]

In [ ]:
for periodo in range(26, 1, -1):
    # Proceso para llegar a la tabla
    proceso = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
    proceso.click()

    presidencial = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
    presidencial.click()

    eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
    eleccion.click()

    # Seleccionar el periodo actual
    periodo_xpath = f'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{periodo}]'
    periodo_element = driver.find_element(By.XPATH, periodo_xpath)
    periodo_element.click()

    # Hacer clic en el botón de buscar
    buscar = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
    buscar.click()

    # Hacer clic en la pestaña de resultados
    resultados = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
    resultados.click()

    # Obtener el HTML de la tabla
    table_path = driver.find_element(By.XPATH, ' ')  # Añade el XPATH de la tabla aquí
    table_html = table_path.get_attribute('innerHTML')

    # Guardar la información en un archivo
    with open(f'periodo_{periodo}_table.html', 'w', encoding='utf-8') as file:
        file.write(table_html)